In [1]:
import logging
from pathlib import Path
from typing import Any, Iterable

In [2]:
from docling_core.types.doc import (
    DoclingDocument,
    NodeItem,
    PictureClassificationClass,
    PictureClassificationData,
    PictureItem,
)

/home/adiazcan/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.models.base_model import BaseEnrichmentModel
from docling.pipeline.standard_pdf_pipeline import StandardPdfPipeline

In [4]:
class ExamplePictureClassifierPipelineOptions(PdfPipelineOptions):
    do_picture_classifer: bool = True

In [5]:
class ExamplePictureClassifierEnrichmentModel(BaseEnrichmentModel):

    def __init__(self, enabled: bool):
        self.enabled = enabled

    def is_processable(self, doc: DoclingDocument, element: NodeItem) -> bool:
        return self.enabled and isinstance(element, PictureItem)

    def __call__(
        self, doc: DoclingDocument, element_batch: Iterable[NodeItem]
    ) -> Iterable[Any]:
        if not self.enabled:
            return

        for element in element_batch:
            assert isinstance(element, PictureItem)

            # uncomment this to interactively visualize the image
            # element.get_image(doc).show()

            element.annotations.append(
                PictureClassificationData(
                    provenance="example_classifier-0.0.1",
                    predicted_classes=[
                        PictureClassificationClass(class_name="dummy", confidence=0.42)
                    ],
                )
            )

            yield element

In [6]:
class ExamplePictureClassifierPipeline(StandardPdfPipeline):

    def __init__(self, pipeline_options: ExamplePictureClassifierPipelineOptions):
        super().__init__(pipeline_options)
        self.pipeline_options: ExamplePictureClassifierPipeline

        self.enrichment_pipe = [
            ExamplePictureClassifierEnrichmentModel(
                enabled=pipeline_options.do_picture_classifer
            )
        ]

    @classmethod
    def get_default_options(cls) -> ExamplePictureClassifierPipelineOptions:
        return ExamplePictureClassifierPipelineOptions()

In [7]:
logging.basicConfig(level=logging.INFO)

input_doc_path = Path("./paciente.pdf")

pipeline_options = ExamplePictureClassifierPipelineOptions()
pipeline_options.images_scale = 2.0
pipeline_options.generate_picture_images = True

doc_converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(
            pipeline_cls=ExamplePictureClassifierPipeline,
            pipeline_options=pipeline_options,
        )
    }
)
result = doc_converter.convert(input_doc_path)

for element, _level in result.document.iterate_items():
    if isinstance(element, PictureItem):
        print(
            f"The model populated the `data` portion of picture {element.self_ref}:\n{element.annotations}"
        )

INFO:docling.document_converter:Going to convert document batch...


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

INFO:docling.pipeline.base_pipeline:Processing document paciente.pdf
INFO:docling.document_converter:Finished converting document paciente.pdf in 96.43 sec.


The model populated the `data` portion of picture #/pictures/0:
[PictureClassificationData(kind='classification', provenance='example_classifier-0.0.1', predicted_classes=[PictureClassificationClass(class_name='dummy', confidence=0.42)])]
The model populated the `data` portion of picture #/pictures/1:
[PictureClassificationData(kind='classification', provenance='example_classifier-0.0.1', predicted_classes=[PictureClassificationClass(class_name='dummy', confidence=0.42)])]
The model populated the `data` portion of picture #/pictures/2:
[PictureClassificationData(kind='classification', provenance='example_classifier-0.0.1', predicted_classes=[PictureClassificationClass(class_name='dummy', confidence=0.42)])]
The model populated the `data` portion of picture #/pictures/3:
[PictureClassificationData(kind='classification', provenance='example_classifier-0.0.1', predicted_classes=[PictureClassificationClass(class_name='dummy', confidence=0.42)])]
The model populated the `data` portion of pi